In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
       
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
import os
import subprocess
from six import string_types
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
%matplotlib inline

!pip install --upgrade pip chart_studio
import chart_studio.plotly as py
import plotly.tools as tls
from plotly.offline import iplot
import cv2
from sklearn.feature_extraction.text import CountVectorizer
from operator import itemgetter

In [ ]:

import gc
pal = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

print('# File sizes')
for f in os.listdir('../input/planets-dataset/planet/planet'):
    if not os.path.isdir('../input/planets-dataset/planet/planet/' + f):
        print(f.ljust(30) + str(round(os.path.getsize('../input/planets-dataset/planet/planet/' + f) / 1000000, 2)) + 'MB')
    else:
        sizes = [os.path.getsize('../input/planets-dataset/planet/planet/'+f+'/'+x)/1000000 for x in os.listdir('../input/planets-dataset/planet/planet/' + f)]
        print(f.ljust(30) + str(round(sum(sizes), 2)) + 'MB' + ' ({} files)'.format(len(sizes)))


In [ ]:
!ls -lha ../input/planets-dataset/planet/planet

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input/planets-dataset/planet/planet"]).decode("utf8"))

In [ ]:
PLANET_KAGGLE_ROOT = os.path.abspath("../input/")
PLANET_KAGGLE_JPEG_DIR = os.path.join(PLANET_KAGGLE_ROOT, '../input/planets-dataset/planet/planet/train-jpg/')
PLANET_KAGGLE_LABEL_CSV = os.path.join(PLANET_KAGGLE_ROOT, '../input/planets-dataset/planet/planet/train_classes.csv')
assert os.path.exists(PLANET_KAGGLE_ROOT)
assert os.path.exists(PLANET_KAGGLE_JPEG_DIR)
assert os.path.exists(PLANET_KAGGLE_LABEL_CSV)

In [ ]:
!ls -lha ../input/planets-dataset/planet/planet/train_classes.csv

In [ ]:
labels_df = pd.read_csv(PLANET_KAGGLE_LABEL_CSV)
labels_df.head()

In [ ]:
labels_df.describe()

In [ ]:
len(labels_df['tags'].unique()) # number of categories

In [ ]:
labels_df.shape # number of individuals

In [ ]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [ ]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
labels_df.head()

In [ ]:
#labels = np.array(labels_df['tags'])

#vect = CountVectorizer()
#vect.fit(labels)
#vect.get_feature_names()

#labels_dtm = vect.transform(labels)
#df_labels = pd.DataFrame(labels_dtm.toarray(), columns = vect.get_feature_names())
#print(df_labels.head())
#print(labels_dtm[:10])

#print(len(df_labels))
#df_labels['image_name'] = labels_df['image_name']
#print(df_labels.head())
#print(df_labels.columns.values)

 #create a dict to collect total values of each class of label
#amazon_condition = {}
#for col in df_labels.columns.values:
    #print(col)
    
    #z = df_labels.groupby([col])[col].count().astype(int)
    #print(type(z))
    #print(col, [np.array(k.astype(int)) for l, k in enumerate(z)])
    #print([z])
    #amazon_condition[col] = 0
    #for i, j in enumerate(z):
        #if i != 0:
            #print ('j =', j) 
            #amazon_condition[col] += j
#print(amazon_condition)

#amazon_condition_labels = [x for x in amazon_condition.keys()]
#amazon_condition_values = [x for x in amazon_condition.values()]

#print(amazon_condition_labels)
#print(amazon_condition_values)


#fig, ax = plt.subplots()

#N = len(amazon_condition_labels)
#ind = np.arange(N)    # the x locations for the groups
#width = 0.5       # the width of the bars

#p1 = ax.bar(ind, amazon_condition_values, width, color='r')
#ax.set_ylabel('Count', size=20)
#ax.set_xlabel('Amazon Condition', size=20)
#ax.set_title('Count by label type', size = 25)
#ax.set_xticks(ind + width/2.)
#ax.set_xticklabels(amazon_condition_labels)

#plotly_fig = tls.mpl_to_plotly( fig )

# For Legend
#plotly_fig["layout"]["showlegend"] = True
#plotly_fig["data"][0]["name"] = "Count"
#plotly_fig["data"][1]["name"] = "Test"

#for tick in ax.get_xmajorticklabels():
    #tick.set_rotation(45)
    #tick.set_horizontalalignment("right")
    #tick.set_fontsize(15)

    
#for tick in ax.get_ymajorticklabels():
    #tick.set_rotation(45)
    #tick.set_horizontalalignment("right")
    #tick.set_fontsize(15)
    
#plt.show()

In [ ]:
labels = labels_df['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1

data=[go.Bar(x=list(counts.keys()), y=list(counts.values()))]
layout=dict(height=800, width=800, title='Distribution of training labels')
fig=dict(data=data, layout=layout)
py.iplot(data, filename='train-label-dist')




In [ ]:
# Co-occurence Matrix
com = np.zeros([len(counts)]*2)
for i, l in enumerate(list(counts.keys())):
    for i2, l2 in enumerate(list(counts.keys())):
        c = 0
        cy = 0
        for row in labels.values:
            if l in row:
                c += 1
                if l2 in row: cy += 1
        com[i, i2] = cy / c

data=[go.Heatmap(z=com, x=list(counts.keys()), y=list(counts.keys()))]
layout=go.Layout(height=800, width=800, title='Co-occurence matrix of training labels')
fig=dict(data=data, layout=layout)
py.iplot(data, filename='train-com')

In [ ]:
def make_cooccurence_matrix(labels):
    numeric_df = labels_df[labels]; 
    c_matrix = numeric_df.T.dot(numeric_df)
    sns.heatmap(c_matrix)
    return c_matrix
    
# Compute the co-ocurrence matrix
make_cooccurence_matrix(label_list)

In [ ]:
# plot the first 9 images in the planet dataset
from matplotlib.image import imread

# define location of dataset
folder = PLANET_KAGGLE_JPEG_DIR

# plot first few images
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	# define filename
	PLANET_KAGGLE_LABEL_CSV = folder + 'train_' + str(i) + '.jpg'
	# load image pixels
	image = imread(PLANET_KAGGLE_LABEL_CSV)
	# plot raw pixel data
	plt.imshow(image)
    
# show the figure
plt.show()

                   

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed

# create a mapping of tags to integers given the loaded mapping file
def create_tag_mapping(labels_df):
	# create a set of all known tags
	labels = set()
	for i in range(len(labels_df)):
		# convert spaced separated tags into an array of tags
		tags = labels_df['tags'][i].split(' ')
		# add tags to the set of known labels
		labels.update(tags)
	# convert set of labels to a list to list
	labels = list(labels)
	# order set alphabetically
	labels.sort()
	# dict that maps labels to integers, and the reverse
	labels_map = {labels[i]:i for i in range(len(labels))}
	inv_labels_map = {i:labels[i] for i in range(len(labels))}
	return labels_map, inv_labels_map

# create a mapping of filename to a list of tags
def create_file_mapping(labels_df):
	mapping = dict()
	for i in range(len(labels_df)):
		name, tags = labels_df['image_name'][i], labels_df['tags'][i]
		mapping[name] = tags.split(' ')
	return mapping
 
# create a one hot encoding for one list of tags
def one_hot_encode(tags, mapping):
	# create empty vector
	encoding = zeros(len(mapping), dtype='uint8')
	# mark 1 for each tag in the vector
	for tag in tags:
		encoding[mapping[tag]] = 1
	return encoding
 
# load all images into memory
def load_dataset(path, file_mapping, tag_mapping):
	photos, targets = list(), list()
	# enumerate files in the directory
	for filename in listdir(folder):
		# load image
		photo = load_img(path + filename, target_size=(32,32))
		# convert to numpy array
		photo = img_to_array(photo, dtype='uint8')
		# get tags
		tags = file_mapping[filename[:-4]]
		# one hot encode tags
		target = one_hot_encode(tags, tag_mapping)
		# store
		photos.append(photo)
		targets.append(target)
	X = asarray(photos, dtype='uint8')
	y = asarray(targets, dtype='uint8')
	return X, y
 
# load the mapping file
#os.chdir("/input/planets-dataset/planet/planet/train_classes.csv")
filename = '../input/planets-dataset/planet/planet/train_classes.csv'
labels_df = pd.read_csv(filename, encoding='latin1')

# create a mapping of tags to integers
mapping, inv_mapping = create_tag_mapping(labels_df)
print(len(mapping))
print(mapping)

# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(labels_df)

# create a mapping of filenames to tag lists
file_mapping = create_file_mapping(labels_df)

# load the jpeg images
folder = '../input/planets-dataset/planet/planet/train-jpg/'
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)

# save both arrays to one file in compressed format
savez_compressed('planet_data.npz', X, y)
    

In [ ]:

from numpy import load
from numpy import ones
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
import sys
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras import backend
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dropout


In [ ]:
# load train and test dataset
def load_dataset():
	# load dataset
	data = load('planet_data.npz')
	X, y = data['arr_0'], data['arr_1']
    #print('Loaded: ', X.shape, y.shape)
    
	# separate into train and test datasets
	X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)
	print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
	return X_train, Y_train, X_test, Y_test

# load dataset
X_train, Y_train, X_test, Y_test = load_dataset()

# make all one predictions
Y_train_pred = asarray([ones(Y_train.shape[1]) for _ in range(Y_train.shape[0])])
Y_test_pred = asarray([ones(Y_test.shape[1]) for _ in range(Y_test.shape[0])])

# evaluate predictions
train_score = fbeta_score(Y_train, Y_train_pred, 2, average='samples')
test_score = fbeta_score(Y_test, Y_test_pred, 2, average='samples')
print('All Ones: train=%.3f, test=%.3f' % (train_score, test_score))

In [ ]:
# vgg16 with fine-tuning and data augmentation 
# load train and test dataset
def load_dataset():
	# load dataset
	data = load('planet_data.npz')
	X, y = data['arr_0'], data['arr_1']
    #print('Loaded: ', X.shape, y.shape)
    
	# separate into train and test datasets
	X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)
	print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
	return X_train, Y_train, X_test, Y_test

# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score 

# define cnn model
def define_model(in_shape=(32, 32, 3), out_shape=17):
	# load model
	model = VGG16(include_top=False, input_shape=in_shape)
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# allow last vgg block to be trainable
	model.get_layer('block5_conv1').trainable = True
	model.get_layer('block5_conv2').trainable = True
	model.get_layer('block5_conv3').trainable = True
	model.get_layer('block5_pool').trainable = True
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(out_shape, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	plt.subplot(211)
	plt.title('Cross Entropy Loss')
	plt.plot(history.history['loss'], color='blue', label='train')
	plt.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	plt.subplot(212)
	plt.title('Fbeta')
	plt.plot(history.history['fbeta'], color='blue', label='train')
	plt.plot(history.history['val_fbeta'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	plt.savefig(filename + '_plot.png')
	plt.close()
    
 # run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	X_train, Y_train, X_test, Y_test = load_dataset()
	# create data generator
	train_datagen = ImageDataGenerator(featurewise_center=True, horizontal_flip=True, vertical_flip=True, rotation_range=90)
	test_datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	train_datagen.mean = [123.68, 116.779, 103.939]
	test_datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterators
	train_it = train_datagen.flow(X_train, Y_train, batch_size=128)
	test_it = test_datagen.flow(X_test, Y_test, batch_size=128)
	# define model
	model = define_model()
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=0)
	# evaluate model
	loss, fbeta = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))
	# learning curves
	summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

In [ ]:
# make a prediction for a new image

# save model
model.save('final_model.h5')

cp train-jpg/train_1.jpg ./sample_image.jpg

# create a mapping of tags to integers given the loaded mapping file
#def create_tag_mapping(labels_df):
	# create a set of all known tags
	#labels = set()
	#for i in range(len(mapping_csv)):
		# convert spaced separated tags into an array of tags
		#tags = mapping_csv['tags'][i].split(' ')
		# add tags to the set of known labels
		#labels.update(tags)
	# convert set of labels to a list to list
	#labels = list(labels)
	# order set alphabetically
	#labels.sort()
	# dict that maps labels to integers, and the reverse
	#labels_map = {labels[i]:i for i in range(len(labels))}
	#inv_labels_map = {i:labels[i] for i in range(len(labels))}
	#return labels_map, inv_labels_map
 
# convert a prediction to tags
def prediction_to_tags(inv_mapping, prediction):
	# round probabilities to {0, 1}
	values = prediction.round()
	# collect all predicted tags
	tags = [inv_mapping[i] for i in range(len(values)) if values[i] == 1.0]
	return tags
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(32, 32))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 32, 32, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 
# load an image and predict the class
def run_example(inv_mapping):
	# load the image
	img = load_image('sample_image.jpg')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
	# map prediction to tags
	tags = prediction_to_tags(inv_mapping, result[0])
	print(tags)
 
# load the mapping file
#filename = 'train_v2.csv'
mapping_csv = read_csv(filename)
# create a mapping of tags to integers
_, inv_mapping = create_tag_mapping(labels_df)
# entry point, run the example
run_example(inv_mapping)